# Aula 08: Modelagem para Classificação

*   **Professor:** Paulo Rogério Pires Manseira
*   **Alunos:** Maruan Biasi El Achkar e Ricardo Falcão Schilieper
*   **GitHub:** github.com/manseiracredit

---

# Importar Dataset - Telco Customer Churn
**Fonte:** https://www.kaggle.com/datasets/blastchar/telco-customer-churn

In [82]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("blastchar/telco-customer-churn")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'telco-customer-churn' dataset.
Path to dataset files: /kaggle/input/telco-customer-churn


# Criar Dataframe

In [83]:
import os
import pandas as pd

path_csv = os.path.join(path, 'WA_Fn-UseC_-Telco-Customer-Churn.csv')

df = pd.read_csv(path_csv)

display(df.head(3))

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes


# Parte 1: Preparação dos Dados para Modelagem

## 1.1 Seleção de Features (Variáveis Independentes) e Target (Variável-Alvo)


* Com base na sua conclusão do Aula 07, crie uma lista com os nomes das colunas que você considera mais importantes:

### Lista de variáveis promissoras a partir da análise gráfica no EDA:

* tenure
* Contract
* SeniorCitizen
* MonthlyCharges
* PaymentMethod
* TotalCharges
* Partner
* Dependents

In [84]:
features = ["tenure", "Contract", "SeniorCitizen", "MonthlyCharges", "PaymentMethod", "TotalCharges", "Partner", "Dependents"]
target = "Churn"

x = df[features]
y = df[target]

display(x.head(3))
display(y.head(3))

,tenure,Contract,SeniorCitizen,MonthlyCharges,PaymentMethod,TotalCharges,Partner,Dependents
0,1,Month-to-month,0,29.85,Electronic check,29.85,Yes,No
1,34,One year,0,56.95,Mailed check,1889.5,No,No
2,2,Month-to-month,0,53.85,Mailed check,108.15,No,No


,Churn
0,No
1,No
2,Yes


## 1.2 Codificação de Variáveis Categóricas (One-Hot Encoding)

In [85]:
x_encoded = pd.get_dummies(x, drop_first=True)


display(x_encoded.head(3))
display(y.head(3))

print("Shape original de X:")
display(x.shape)
print("--------------")
print("Shape tratato de X:")
display(x_encoded.shape)

x = x_encoded

,tenure,SeniorCitizen,MonthlyCharges,Contract_One year,Contract_Two year,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,TotalCharges_100.2,TotalCharges_100.25,...,TotalCharges_996.85,TotalCharges_996.95,TotalCharges_997.65,TotalCharges_997.75,TotalCharges_998.1,TotalCharges_999.45,TotalCharges_999.8,TotalCharges_999.9,Partner_Yes,Dependents_Yes
0,1,0,29.85,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,34,0,56.95,True,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
2,2,0,53.85,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False


,Churn
0,No
1,No
2,Yes


Shape original de X:


(7043, 8)

--------------
Shape tratato de X:


(7043, 6540)

## Pontos de Parada para Discussão em Dupla (Reflexão):

1. "Por que não incluímos todas as colunas do dataset como features? Qual é a
desvantagem de usar variáveis que a nossa EDA mostrou não ter relação com o Churn?"                    
**Resp.:** Pois as colunas que não tem relação com nossa variável alvo só servem para criar *noise/ruído* nos modelos de predição, além disso, deixa o treinamento muito mais demorado.

2. "Ao usar  pd.get_dummies() , uma coluna como  Contract  (com 3 categorias) se
transforma em 2 ou 3 novas colunas. Por que isso acontece? O que os valores 0 e 1 nessas novas colunas representam?"                
**Resp.:** Pois a coluna Contract possui 3 tipos diferentes de valores, ou seja, na hora de transformar isso em um boolean, que aceita apenas dois valores (sim ou não) é necessário separar os três valores em três colunas diferentes, para que o boolean consiga representar todos eles. Ou seja, em vez de uma coluna dizendo qual o tipo de contrato, temos três colunas, cada uma dizendo que o tipo de contrato é ou não é aquele.

-----------

# Parte 2: Treino e Teste - A Regra de Ouro do Machine Learning

In [86]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
      x,
      y,
      test_size=0.3,
      random_state=42,
      stratify=y
  )

In [87]:
print(f"Tamanho do conjunto de treino (X_train): {x_train.shape[0]} clientes")
print(f"Tamanho do conjunto de teste (X_test): {x_test.shape[0]} clientes")

Tamanho do conjunto de treino (X_train): 4930 clientes
Tamanho do conjunto de teste (X_test): 2113 clientes


In [88]:
print("\nProporção de Churn no y_train:")
print(y_train.value_counts(normalize=True))
print("\nProporção de Churn no y_test:")
print(y_test.value_counts(normalize=True))


Proporção de Churn no y_train:
Churn
No     0.734686
Yes    0.265314
Name: proportion, dtype: float64

Proporção de Churn no y_test:
Churn
No     0.734501
Yes    0.265499
Name: proportion, dtype: float64


# Parte 3: Construindo e Avaliando o Modelo de Classificação

In [89]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

decisiontree = DecisionTreeClassifier(
    max_depth = 6,
    random_state = 42
    )

In [90]:
decisiontree.fit(x_train, y_train)

DecisionTreeClassifier(max_depth=6, random_state=42)

In [91]:
previsoes = decisiontree.predict(x_test)

In [92]:
accuracy = accuracy_score(y_test, previsoes)
print(f"Accuracy: {accuracy:.2%}")

Accuracy: 77.61%


## Pontos de Parada para Discussão em Dupla (Reflexão):

1. Nossa acurácia foi de  77.61% . Isso é um bom resultado? Comparem com a proporção de "Não Churn" que vocês viram na EDA. Nosso modelo é mais inteligente do que simplesmente chutar "Não Churn" para todos os clientes?

**Resp.:** Sim, é um bom resultado. A porcentagem de clientes que não deram churn, de acordo com o EDA, é de 73.46%, isso significa que nosso modelo foi mais eficaz do que simplesmente chutar.

2. O que significa  max_depth=4 ? Se aumentarmos para  max_depth=10 , a acurácia nos dados de teste necessariamente aumentaria? Qual o risco de criar uma árvore muito complexa?

**Resp.:** Significa a quantidade de "camadas" que nossa árvore vai ter, ou como o nome sugere, sua profundidade. No nosso caso, testamos com alguns valores diferentes e encontramos a maior acurácia (77.61%) utilizando um *max_depth=6*. Não, se mudarmos para 10, não necessariamente aumentaria, na verdade, de acordo com nossos testes, a acurácia diminui com um max_depth de 10, isso, por causa do overfitting, que é um modelo especialista demais nos dados utilizados no treinamento, é como se o modelo tivesse decorado os dados que já viu, se tornando ruim na hora de analizar novos dados.

# Conclusão Final

Concluimos que o modelo criado é útil para nosso negócio, mas acreditamos que uma acurácia melhor pode ser obtiva através de outros modelos, até mesmo versões mais otimizadas do modelo de árvore como RandomForest, CatBoost, XGBoost e LightGBM.

# Teste com o modelo CatBoost

In [93]:
!pip install catboost
import catboost as cb

In [94]:
catboost = cb.CatBoostClassifier(
    random_state=42,
    verbose=0
)

catboost.fit(x_train, y_train)

catboost_previsoes = catboost.predict(x_test)

catboost_accuracy = accuracy_score(y_test, catboost_previsoes)
print(f"Accuracy do CatBoost: {catboost_accuracy:.2%}")

Accuracy do CatBoost: 79.79%


# Conclusão CatBoost

Nossa hipótese de que o CatBosst poderia trazer uma acurácia melhor estava correta. Conseguimos um aumento de 2,18 pontos percentuais comparado ao modelo tradicional de árvore. Acreditamos que essa acurácia pode ser melhorada mais ainda utilizando modelos mais compatíveis com o nosso cenário e também através de otimização de parametros.